In [127]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [128]:
# predecir la gravedad de un accidente
#

datos_siniestros = pd.read_excel("2015_2019_siniestralidad_vial.xlsx", sheet_name="SINIESTROS")
datos_siniestros.head()




,CODIGO_SINIESTRO,FECHA,HORA,GRAVEDAD,CLASE,CHOQUE_CON,OBJETO_FIJO,DIRECCION,TOTAL_MUERTOS,TOTAL_HERIDOS,LOCALIDAD,DISENO_LUGAR
0,4401423,01/01/2015,09:45:00,Con Heridos,Choque,Vehiculo,NaN,CL 66A-KR 76 02,0,2,Engativá,Interseccion
1,4401425,01/01/2015,13:25:00,Solo Daños,Choque,Vehiculo,NaN,CL 143A-KR 142 02,0,0,Suba,Tramo de Via
2,4401430,01/01/2015,07:15:00,Con Heridos,Volcamiento,NaN,NaN,KR 19D-CL 62 S 02,0,1,Ciudad Bolívar,Tramo de Via
3,4401433,01/01/2015,20:30:00,Solo Daños,Choque,Vehiculo,NaN,AK 7-CL 43 33,0,0,Chapinero,Tramo de Via
4,4401437,01/01/2015,12:50:00,Solo Daños,Choque,Vehiculo,NaN,DG 77A-AK 18D 02,0,0,Ciudad Bolívar,Tramo de Via


Prediccion: Gravedad

In [129]:
# se recorren la columna Gravedad, al encontrar solo daños, no será de gravedad, en caso de tener heridos o muertos serlo si lo será
datos_siniestros['GRAVEDAD'] = datos_siniestros['GRAVEDAD'].map({'Solo Daños': 0, 'Con Heridos': 1 , "Con Muertos": 1})

In [130]:
datos_siniestros["GRAVEDAD"].head()

0    1
1    0
2    1
3    0
4    0
Name: GRAVEDAD, dtype: int64

Variables Independientes: Diseño Lugar, Localidad, ChoqueCon, Clase
Variable Dependiente: Gravedad

In [131]:
# se crea el dataframe para predecir
predicciónGravedad = datos_siniestros.filter(["DISENO_LUGAR","LOCALIDAD","CLASE","CHOQUE_CON","GRAVEDAD"],axis = 1)
predicciónGravedad.head()

,DISENO_LUGAR,LOCALIDAD,CLASE,CHOQUE_CON,GRAVEDAD
0,Interseccion,Engativá,Choque,Vehiculo,1
1,Tramo de Via,Suba,Choque,Vehiculo,0
2,Tramo de Via,Ciudad Bolívar,Volcamiento,NaN,1
3,Tramo de Via,Chapinero,Choque,Vehiculo,0
4,Tramo de Via,Ciudad Bolívar,Choque,Vehiculo,0


In [132]:
# Se convierten variables categoricas a variable numerica ordinal
predicciónGravedad["DISENO_LUGAR"] = predicciónGravedad["DISENO_LUGAR"].astype('category')
predicciónGravedad["DISENO_LUGAR"] = predicciónGravedad["DISENO_LUGAR"].cat.codes
predicciónGravedad["LOCALIDAD"] = predicciónGravedad["LOCALIDAD"].astype('category')
predicciónGravedad["LOCALIDAD"] = predicciónGravedad["LOCALIDAD"].cat.codes
predicciónGravedad["CLASE"] = predicciónGravedad["CLASE"].astype('category')
predicciónGravedad["CLASE"] = predicciónGravedad["CLASE"].cat.codes
predicciónGravedad["CHOQUE_CON"] = predicciónGravedad["CHOQUE_CON"].astype('category')
predicciónGravedad["CHOQUE_CON"] = predicciónGravedad["CHOQUE_CON"].cat.codes

In [133]:
predicciónGravedad.head()

,DISENO_LUGAR,LOCALIDAD,CLASE,CHOQUE_CON,GRAVEDAD
0,2,5,3,3,1
1,9,14,3,3,0
2,9,4,6,-1,1
3,9,3,3,3,0
4,9,4,3,3,0


In [134]:
#Se escoge x como todas las columnas excepto la objetivo
X = np.array(predicciónGravedad.drop(['GRAVEDAD'],1))
# variable objetivo
y = np.array(predicciónGravedad['GRAVEDAD'])


In [135]:
#MODELO DE LA REGRESION LOGISTICA

model = linear_model.LogisticRegression()
model.fit(X,y)


LogisticRegression()

In [136]:
predictions = model.predict(X)
print(predictions)

[0 0 1 ... 1 1 0]


In [137]:
model.score(X,y)

0.7812826115784436

In [138]:
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)



In [139]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10, random_state=seed,shuffle=True)
cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(msg)


Logistic Regression: 0.781188 (0.003436)


In [140]:
predictions = model.predict(X_validation)
print(accuracy_score(Y_validation, predictions))



0.7802185130733086


In [141]:
print(confusion_matrix(Y_validation, predictions))


[[21838   919]
 [ 6705  5227]]


In [142]:
print(classification_report(Y_validation, predictions))


              precision    recall  f1-score   support

           0       0.77      0.96      0.85     22757
           1       0.85      0.44      0.58     11932

    accuracy                           0.78     34689
   macro avg       0.81      0.70      0.71     34689
weighted avg       0.79      0.78      0.76     34689



In [143]:
#prediccion
#localidad = 5 Engativa
#Diseño Lugar = 2 Interseccion
#Clase = 3 Choque
#Choque con = 4 Vehiculo
X_new = pd.DataFrame({'DISEÑO_LUGAR': [5], 'LOCALIDAD': [2], 'CLASE': [3], 'CHOQUE_CON': [4]})
model.predict(X_new)


array([0], dtype=int64)